# Running Molecular Dynamics Simulations in OpenMM

## Overview
**Molecular Dynamics Simulation**: simulation of molecules through time via calculations of forces on atoms and their positions based on those forces. The output is a trajectory.

## Steps for running a simulation
1. **Initialization** -- setting up imports, variables, and other system parameters to prepare for the simulation
2. **Minimization** -- calculating the energies of moving molecules slightly to obtain a local "minimum" in energy and eliminate “bad” interatomic contacts
3. **Equilibration** -- a brief MD simulation that prepares our system to our target temperature and desired equilibrium values
4. **Production** -- run the equilibration
5. **Analysis** -- after you have collected your data from the production run, you must analyze the trajectory to draw conclusions

## Our First Simulations
We will now use [OpenMM](https://openmm.org/documentation) to do a molecular dynamics simulation of the ethane and butane molecules we prepared. It’s important to note at this point that molecular dynamics simulations can be performed using a number of softwares. However, we will be running a simulation with a program called OpenMM. OpenMM has the advantage of being scriptable with Python.

---

### Initialization
Once you have OpenMM, we can use it to simulate our molecules. Start in your notebook with imports. Here are the python libraries you will need to run simulations with OpenMM.

In [ ]:
import openmm as mm
from openmm import app, unit

First, we need to read in our structure and our force field. We have to tell the simulation our initial coordinates and the force field we will use. To do this, we use the PDB file we have and the force field file we prepared.

In [ ]:
pdb = app.PDBFile('data/ethane.pdb')
forcefield = app.ForceField('data/ethane.gaff2.xml')

We read in the PDB file to get the initial coordinates in the first command. In the second command we give OpenMM the force field file we prepared for ethane before and add it to our `pdb` variable.

Next, we set up the system for our MD simulation. With the following command, we use the `pdb` variable to create a system. The other arguments to the function say that we are not using a cut-off (discussed more later) and that we want to constrain bonds with hydrogens (this allows us to use a larger timestep).

In [ ]:
system = forcefield.createSystem(pdb.topology, nonbondedMethod=app.NoCutoff, constraints=app.HBonds)

To run a molecular dynamics simulation we have to integrate Newton’s equations of motion. There are different algorithms you might choose to do this, but for this exercise we are going to use a Langevin integrator. Our simulation will be in vacuum at a temperature of 298.15 K. The Langevin integrator is what is called a stochastic integrator. This means that it mimics jostling of air or solvent through random forces. We are using a 5.0 picosecond coupling constant, which is something which controls how often the integrator adds jostling motions.

In [ ]:
integrator = mm.LangevinIntegrator(298.15*unit.kelvin, 5.0/unit.picoseconds, 2.0*unit.femtoseconds)
integrator.setConstraintTolerance(1e-5)

Finally, we initialize the simulation by adding all of the pieces we have prepared:

In [ ]:
platform = mm.Platform.getPlatformByName('Reference')
simulation = app.Simulation(pdb.topology, system, integrator, platform)
simulation.context.setPositions(pdb.positions)

---

### Minimization
Now, we start calculating energies. First we do an energy minimization. An energy minimization just moves the atoms of the molecule slightly to get to a local minimum in energy. We start in this code block by printing the energy before minimization, doing 100 steps of an energy minimization, then printing the new energy. You should see that the energy decreases:

In [ ]:
print('Minimizing...')

st = simulation.context.getState(getPositions=True,getEnergy=True)
print(F"Potential energy before minimization is {st.getPotentialEnergy()}")

simulation.minimizeEnergy(maxIterations=100)

st = simulation.context.getState(getPositions=True,getEnergy=True)
print(F"Potential energy after minimization is {st.getPotentialEnergy()}")

You can’t see it from this code, but the atom positions have changed slightly to cause this change in energy.

---

### Equilibration
Next, we run an equilibration. The purpose of this equilibration is to get our system to our target temperature and to get the system equilibrated and ready for our production run.

In [ ]:
from sys import stdout

print('Equilibrating...')

simulation.reporters.append(app.StateDataReporter(stdout, 100, step=True, 
    potentialEnergy=True, temperature=True, separator=','))
simulation.context.setVelocitiesToTemperature(150.0*unit.kelvin)
simulation.step(2500)

The first command in this section sets up what information OpenMM will report to us as the simulation runs. We are asking for the potential energy, and temperature every 100 timesteps. By putting `stdout` as the first argument, we are telling the program to just print the information. Note that `stdout` comes from the built in Python module `sys`. If we wanted the information in a file instead, you would put the file name.

The second command sets the initial velocities of the system to a temperature equivalent of 150 K. Then, we integrate for 2,500 steps to allow the system to equilibrate.

---

### Production
This next block of code is a longer block of simulation called the ‘production run’. We’re also added a timer to this code so we can see how long it took our simulation to run.

In [ ]:
import time as time

print('Running Production...')

# Begin timer
tinit=time.time()

# Clear simulation reporters
simulation.reporters.clear()

# Reinitialize simulation reporters. We do this because we want different information printed from the production run than the equilibration run.
# output basic simulation information below every 250000 steps - (which is equal to 2 fs(250,000) = 500,000 fs = 500 ps)
simulation.reporters.append(app.StateDataReporter(stdout, 250000, 
    step=True, time=True, potentialEnergy=True, temperature=True, 
    speed=True, separator=','))

# write out a trajectory (i.e., coordinates vs. time) to a DCD
# file every 100 steps - 0.2 ps
simulation.reporters.append(app.DCDReporter('ethane_sim.dcd', 100))

# run the simulation for 1.0x10^7 steps - 20 ns
simulation.step(10000000)

# End timer
tfinal=time.time()
print('Done!')
print('Time required for simulation:', tfinal-tinit, 'seconds')

After executing this cell in your notebook, you should see an output which gives you the step number, simulation time, potential energy, temperature, and “speed” for steps in the simulation. The spacing of theses is set in the `simulation.reporters` step where we indicated we wanted information printed every 250,000 timesteps.

The “speed” is reported in “ns/day” or “nanoseconds/day”. This is a commonly used unit to report how quickly simulations run. It tells you how much simulation time in nanoseconds would pass for 24 hours of computation time. For example, if a simulation is running at 2 ns/day, enough timesteps would be calculated in one day to make 2 nanoseconds of simulation time. If we were using our 2 fs timestep, this would mean that the computer calculated 1,000,000 timesteps over 24 hours.

---

## Your Turn
Make a copy of the code you wrote to run your ethane simulation and modify it to:
1. Read in the files `butane.gaff2.xml` and `butane.pdb`
2. Carry out a 10 ps MD simulation to bring the butane molecule to an equilibrium temperature of 298 K in which output is printed every 0.5 ps (Leave the minimization portion beforehand unchanged.)
3. Carry out a 40 ns MD simulation at 298 K in which output is printed every 1 ns and structures are (still) saved every 0.2 ps into a file called `butane_sim.dcd`.

In [ ]:
### your code here!!!

---

## Analysis
Now that we’ve performed our computer experiment, it is time to analyze the data we have collected. The main type of data you have collected through this simulation is information on atom positions, or the system trajectory.

As part of our production simulation, we set up a reporter to record atomic positions. The code below shows that code from your previous script, **you do not need to execute it.**

In [ ]:
simulation.reporters.append(app.DCDReporter('ethane_sim.dcd', 100))

This reporter saved the atomic positions for us every 100 timesteps in a file called `ethane_sim.dcd`. The DCD file format is a binary file (instead of being a text file), so you cannot open it and look at it. However, we will be using certain libraries to analyze and view the file’s contents. If you’ve run your simulation, you should have the file `ethane_sim.dcd` in the same folder as your Jupyter notebook.

First, we will need to make sure we have a few more Python libraries installed which can help us with analysis. We will use a library called **nglview** to visualize the trajectory, and a library called **MDTraj** to analyze the trajectory. Before opening a new notebook for analysis, you may need to install **nglview** and **MDTraj**.

Type the following in your *terminal* to install **nglview** and **MDTraj**:

In [ ]:
$ conda install -c conda-forge mdtraj nglview

In [ ]:
import mdtraj as md

traj = md.load('ethane_sim.dcd', top='data/ethane.pdb')

The command above reads all of the atomic positions from `ethane_sim.dcd` and keeps track of atom connectivity (topology) which was given in the PDB file. Next, visualize the trajectory using nglview. Nglview has a special function `show_mdtraj` that we can use with our trajectory because it was in a specific format from the MDTraj library.

In [ ]:
import nglview as ngl

visualize = ngl.show_mdtraj(traj)
visualize

This should show you something that looks sort of like a movie of your ethane molecule. These are the atomic positions calculated by OpenMM during the molecular dynamics run. We can now analyze the positions to find out some things about our molecule.

We will use another OpenMM command to pull out our bonds and atoms for analysis

In [ ]:
atoms, bonds = traj.topology.to_dataframe()
atoms

### Analyzing the C-C bond length
Let’s look at what C-C bond lengths our ethane molecule had during the simulation. Before we can measure the bond lengths, we have to decide which atoms from our molecule define the bond angle. You should see a table output from running `atoms`.

We have to pick the atom indices for the C-C bond. An atom’s index is the left-most value in the table above. For our torsion, we’ll measure `C1-C2` the indices for these are `0` and `4`. We use the function `compute_distances` in the MDTraj library to measure the distance between these atoms.

In [ ]:
bond_indices = [0, 4] # atoms to define the bond length
bond_length = md.compute_distances(traj, [bond_indices])

We now have the measurement for this torsion angle in radians for each recorded timestep of the trajectory saved in the array `bond_length`. One way we can examine this data is by plotting it as a histogram using the Python library `matplotlib`.

In [ ]:
import matplotlib.pyplot as plt

bondcounts, binedges, otherstuff = plt.hist(bond_length, bins=120)
plt.title('C-C bond length histogram')
plt.xlabel('Bond length (nm)')
plt.ylabel('Counts')
plt.show()

## Your Turn
A torsion is made up of four atoms which are bonded to each other. Analyze the torsion angle associated with the atoms `H11-C1-C2-H21` for your trajectory. Instead of using the function `compute_distance`, use `compute_dihedrals`. Create a histogram plot of the torsion angles.

First, we need to pick the atom indices of our torsion angle and use the `compute_dihedrals` function to calculate the dihedrals.

In [ ]:
phi_indices = [] # add atoms to define the torsion angle
phi = # use the compute_dihedrals function

print(phi)

We now have the measurement for this torsion angle in radians for each recorded timestep of the trajectory.

Next, we can examine this data by plotting it as a histogram using the Python library `matplotlib`.

In [ ]:
import matplotlib.pyplot as plt

# create a histogram with 90 bins

Credit for this exercise goes to [MolSSI](https://education.molssi.org/mm-tools/02-md-alkanes/index.html).